In [299]:
import h5py
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import shap
import lightgbm as lgb 
import optuna
from optuna.samplers import TPESampler
from optuna.integration import LightGBMPruningCallback
from optuna.pruners import MedianPruner
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import mean_absolute_error 
from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif, mutual_info_regression
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
import xgboost

In [300]:
# read data
def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        filename = name.split('/')[-1]
        return pandas.DataFrame(f[filename][:], dtype=np.float64)

train = load_data('train')
test  = load_data('test')

print (f'Shape of training data set: {train.shape}')
print (f'Shape of test data set: {test.shape}')

all_variables = ['actualInteractionsPerCrossing', 'averageInteractionsPerCrossing', 'correctedActualMu', 'correctedAverageMu', 'correctedScaledActualMu', 'correctedScaledAverageMu', 'NvtxReco', 'p_nTracks', 'p_pt_track', 'p_eta', 'p_phi', 'p_charge', 'p_qOverP', 'p_z0', 'p_d0', 'p_sigmad0', 'p_d0Sig', 'p_EptRatio', 'p_dPOverP', 'p_z0theta', 'p_etaCluster', 'p_phiCluster', 'p_eCluster', 'p_rawEtaCluster', 'p_rawPhiCluster', 'p_rawECluster', 'p_eClusterLr0', 'p_eClusterLr1', 'p_eClusterLr2', 'p_eClusterLr3', 'p_etaClusterLr1', 'p_etaClusterLr2', 'p_phiClusterLr2', 'p_eAccCluster', 'p_f0Cluster', 'p_etaCalo', 'p_phiCalo', 'p_eTileGap3Cluster', 'p_cellIndexCluster', 'p_phiModCalo', 'p_etaModCalo', 'p_dPhiTH3', 'p_R12', 'p_fTG3', 'p_weta2', 'p_Reta', 'p_Rphi', 'p_Eratio', 'p_f1', 'p_f3', 'p_Rhad', 'p_Rhad1', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_TRTPID', 'p_TRTTrackOccupancy', 'p_numberOfInnermostPixelHits', 'p_numberOfPixelHits', 'p_numberOfSCTHits', 'p_numberOfTRTHits', 'p_numberOfTRTXenonHits', 'p_chi2', 'p_ndof', 'p_SharedMuonTrack', 'p_E7x7_Lr2', 'p_E7x7_Lr3', 'p_E_Lr0_HiG', 'p_E_Lr0_LowG', 'p_E_Lr0_MedG', 'p_E_Lr1_HiG', 'p_E_Lr1_LowG', 'p_E_Lr1_MedG', 'p_E_Lr2_HiG', 'p_E_Lr2_LowG', 'p_E_Lr2_MedG', 'p_E_Lr3_HiG', 'p_E_Lr3_LowG', 'p_E_Lr3_MedG', 'p_ambiguityType', 'p_asy1', 'p_author', 'p_barys1', 'p_core57cellsEnergyCorrection', 'p_deltaEta0', 'p_deltaEta2', 'p_deltaEta3', 'p_deltaPhi0', 'p_deltaPhi1', 'p_deltaPhi2', 'p_deltaPhi3', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhiRescaled0', 'p_deltaPhiRescaled1', 'p_deltaPhiRescaled3', 'p_e1152', 'p_e132', 'p_e235', 'p_e255', 'p_e2ts1', 'p_ecore', 'p_emins1', 'p_etconeCorrBitset', 'p_ethad', 'p_ethad1', 'p_f1core', 'p_f3core', 'p_maxEcell_energy', 'p_maxEcell_gain', 'p_maxEcell_time', 'p_maxEcell_x', 'p_maxEcell_y', 'p_maxEcell_z', 'p_nCells_Lr0_HiG', 'p_nCells_Lr0_LowG', 'p_nCells_Lr0_MedG', 'p_nCells_Lr1_HiG', 'p_nCells_Lr1_LowG', 'p_nCells_Lr1_MedG', 'p_nCells_Lr2_HiG', 'p_nCells_Lr2_LowG', 'p_nCells_Lr2_MedG', 'p_nCells_Lr3_HiG', 'p_nCells_Lr3_LowG', 'p_nCells_Lr3_MedG', 'p_pos', 'p_pos7', 'p_poscs1', 'p_poscs2', 'p_ptconeCorrBitset', 'p_ptconecoreTrackPtrCorrection', 'p_r33over37allcalo', 'p_topoetconeCorrBitset', 'p_topoetconecoreConeEnergyCorrection', 'p_topoetconecoreConeSCEnergyCorrection', 'p_weta1', 'p_widths1', 'p_widths2', 'p_wtots1', 'p_e233', 'p_e237', 'p_e277', 'p_e2tsts1', 'p_ehad1', 'p_emaxs1', 'p_fracs1', 'p_DeltaE', 'p_E3x5_Lr0', 'p_E3x5_Lr1', 'p_E3x5_Lr2', 'p_E3x5_Lr3', 'p_E5x7_Lr0', 'p_E5x7_Lr1', 'p_E5x7_Lr2', 'p_E5x7_Lr3', 'p_E7x11_Lr0', 'p_E7x11_Lr1', 'p_E7x11_Lr2', 'p_E7x11_Lr3', 'p_E7x7_Lr0', 'p_E7x7_Lr1' ]

all = train[all_variables]

electron_truth = train['Truth']
y = train['p_truth_E']
y = y[electron_truth==1]
X = train[all_variables]
X = X[electron_truth==1]
y = y.to_numpy(dtype='float32')

# print(y)
# prepocessing

norm = preprocessing.MinMaxScaler()
sc_X = preprocessing.StandardScaler()
sc_y = preprocessing.StandardScaler()
X = sc_X.fit_transform(X)
y = np.reshape(y, (-1,1))
y = sc_y.fit_transform(y)
y = y.ravel()

print(y)

X = pd.DataFrame(X, columns=all_variables)
# y = pd.DataFrame(y, columns=['p_truth_E'])


print (f'Shape of X: {X.shape}')
print (f'Shape of y: {y.shape}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)




Shape of training data set: (162500, 166)
Shape of test data set: (160651, 164)
[-0.41764802 -0.01699412  0.01510229 ... -0.29044643 -0.21449289
 -1.1436536 ]
Shape of X: (121495, 160)
Shape of y: (121495,)


In [301]:
# feature_selection
def selection_features(X_train, y_train, X_test):
    select = SelectKBest(score_func=mutual_info_regression, k=12)
    select.fit(X_train, y_train)
    
    return select

In [302]:
# select = selection_features(X_train, y_train, X_test)
# X = pd.DataFrame(X)
# names = X.columns.values[select.get_support()]
# scores = select.scores_[select.get_support()]
# names_scores = list(zip(names, scores))
# ns_df = pd.DataFrame(data = names_scores, columns=['Feature_names', 'Feature_scores'])
# #Sort the dataframe for better visualization
# ns_df_sorted = ns_df.sort_values(['Feature_names', 'Feature_scores'], ascending = [False, True])
# print(ns_df_sorted)

In [303]:
# def get_feature_importance():
#     model = SelectKBest(mutual_info_regression, k=12)#选择k个最佳特征
#     X_new = model.fit_transform(X, y)
#     #feature_data是特征数据，label_data是标签数据，该函数可以选择出k个特征 
 
#     print("model shape: ",X_new.shape)
 
#     scores = model.scores_
#     print('model scores:', scores)  # 得分越高，特征越重要
 
#     p_values = model.pvalues_
#     print('model p-values', p_values)  # p-values 越小，置信度越高，特征越重要
 
#     # 按重要性排序，选出最重要的 k 个
#     indices = np.argsort(scores)[::-1]
#     k_best_features = list(X.columns.values[indices[0:12]])
 
#     print('k best features are: ',k_best_features)
    
#     return k_best_features


In [304]:
kbest_variables = ['p_eCluster',
 'p_rawECluster',
 'p_eAccCluster',
 'p_ecore',
 'p_E7x11_Lr2',
 'p_E7x7_Lr2',
 'p_e277',
 'p_eClusterLr2',
 'p_E5x7_Lr2',
 'p_e255',
 'p_e237',
 'p_e235']


sc_X_kbest = preprocessing.StandardScaler()
X_kbest = train[kbest_variables]
X_kbest = X_kbest[electron_truth==1]
X_kbest = sc_X_kbest.fit_transform(X_kbest)
X_kbest = pd.DataFrame(X_kbest, columns=kbest_variables)

sc_y_kbest = preprocessing.StandardScaler()
y_kbest = train['p_truth_E']
y_kbest = y_kbest[electron_truth==1]
y_kbest = y_kbest.to_numpy(dtype='float32')
y_kbest = np.reshape(y_kbest, (-1,1))
y_kbest = sc_y_kbest.fit_transform(y_kbest)
y_kbest = y_kbest.ravel()

X_kbest_train, X_kbest_test, y_kbest_train, y_kbest_test = train_test_split(X_kbest, y_kbest, test_size=0.2, random_state=12)

sc_input_kbest = preprocessing.StandardScaler()
input_valid_kbest= test[kbest_variables]
input_valid_kbest = sc_input_kbest.fit_transform(input_valid_kbest)
input_valid_kbest = pd.DataFrame(input_valid_kbest, columns=[kbest_variables])

 

In [305]:
shap_variables = ['p_eCluster', 'p_eAccCluster', 'p_nCells_Lr1_HiG', 'p_ecore', 'p_EptRatio', 'p_rawECluster', 'p_d0', 'p_E3x5_Lr1', 'p_nTracks', 'p_deltaEta2', 'p_pt_track', 'p_deltaPhi2' ]

X_shap = train[shap_variables]
X_shap = X_shap[electron_truth==1]
X_shap = sc_X.fit_transform(X_shap)
X_shap = pd.DataFrame(X_shap, columns=shap_variables)

sc_y_shap = preprocessing.StandardScaler()
y_shap = train['p_truth_E']
y_shap = y_shap[electron_truth==1]
y_shap = y_shap.to_numpy(dtype='float32')
y_shap = np.reshape(y_shap, (-1,1))
y_shap = sc_y_shap.fit_transform(y_shap)
y_shap = y_shap.ravel()

X_shap_train, X_shap_test, y_shap_train, y_shap_test = train_test_split(X_shap, y_shap, test_size=0.2, random_state=12)

sc_input_shap = preprocessing.StandardScaler()
input_valid_shap = test[shap_variables]
input_valid_shap = sc_input_shap.fit_transform(input_valid_shap)
input_valid_shap = pd.DataFrame(input_valid_shap, columns=[shap_variables])

In [306]:
lgb_variables = ['p_eCluster', 'p_deltaEta2', 'p_deltaPhiRescaled2', 'p_deltaEta1', 'p_EptRatio', 'p_d0', 'p_deltaPhi2', 'p_sigmad0', 'p_ecore', 'p_qOverP', 'p_pt_track', 'p_DeltaE']
X_lgb = train[lgb_variables]
X_lgb = X_lgb[electron_truth==1]
X_lgb = sc_X.fit_transform(X_lgb)
X_lgb = pd.DataFrame(X_lgb, columns=lgb_variables)

sc_y_lgb = preprocessing.StandardScaler()
y_lgb = train['p_truth_E']
y_lgb= y_lgb[electron_truth==1]
y_lgb = y_lgb.to_numpy(dtype='float32')
y_lgb = np.reshape(y_lgb, (-1,1))
y_lgb = sc_y_lgb.fit_transform(y_lgb)
y_lgb = y_lgb.ravel()

X_lgb_train, X_lgb_test, y_lgb_train, y_lgb_test = train_test_split(X_lgb, y_lgb, test_size=0.2, random_state=12)
sc_input_lgb = preprocessing.StandardScaler()
input_valid_lgb = test[lgb_variables]
input_valid_lgb = sc_input_lgb.fit_transform(input_valid_lgb)
input_valid_lgb = pd.DataFrame(input_valid_lgb, columns=[lgb_variables])

variable_list_shap = pd.DataFrame(shap_variables, columns=['vars'])
variable_list_shap.to_csv('solutions/Regression_HauLamFong_xgboost_VariableList.txt')
variable_list_lgb = pd.DataFrame(lgb_variables, columns=['vars'])
variable_list_lgb.to_csv('solutions/Regression_HauLamFong_lightgbm_VariableList.txt')
variable_list_kbest = pd.DataFrame(kbest_variables, columns=['vars'])
variable_list_kbest.to_csv('solutions/Regression_HauLamFong_neuralnetwork_VariableList.txt')

In [307]:
def feature_importance_lgb():
    gbm = lgb.LGBMRegressor()
    gbm.fit(X_train, y_train)
    gbm.booster_.feature_importance()

    feature_imp_ = pd.DataFrame({'cols':X_train.columns, 'feature_imp':gbm.feature_importances_})
    feature_imp_.loc[feature_imp_.feature_imp > 0].sort_values(by=['feature_imp'], ascending=False)

    return feature_imp_.loc[feature_imp_.feature_imp > 0].sort_values(by=['feature_imp'], ascending=False).head(12)

feature_importance_lgb()

,cols,feature_imp
22,p_eCluster,264
84,p_deltaEta2,180
53,p_deltaPhiRescaled2,131
52,p_deltaEta1,125
17,p_EptRatio,92
14,p_d0,91
88,p_deltaPhi2,86
15,p_sigmad0,79
99,p_ecore,71
12,p_qOverP,66


In [308]:
from lightgbm import LGBMRegressor
# predict

# optimize hyperparameters of lgb

# def objective_lgb(trial):

#     X_lgb_train, X_lgb_test, y_lgb_train, y_lgb_test = train_test_split(X_lgb, y, test_size=0.2, random_state=42)
#     train_data = lgb.Dataset(X_lgb_train, label=y_lgb_train)
#     valid_data = lgb.Dataset(X_lgb_test, label=y_lgb_test)
    
#     boosting_types = ["gbdt", "rf", "dart"]
#     boosting_type = trial.suggest_categorical("boosting_type", boosting_types)

#     params = {
#         'objective': 'regression',
#         'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt", "rf", "dart"]),
#         'max_depth': trial.suggest_int('max_depth', 2, 100),
#         'min_child_samples': trial.suggest_int('min_child_samples', 0, 1000),
#         'metric': 'mae',
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 0.95, step=0.1),
#         'num_leaves': trial.suggest_int('num_leaves', 20, 3000, step=20),
#         'bagging_freq': trial.suggest_categorical('bagging_freq', [1]),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.95, step=0.1),
#         'reg_alpha': trial.suggest_float("reg_alpha", 0, 100, step=0.1),
#         'reg_lambda': trial.suggest_int("reg_lambda", 0, 1000, step=1),
#         'verbosity': -1,
#     }

#     lgbm = LGBMRegressor(**params)
#     lgbm.fit(X_lgb_train, y_lgb_train, eval_set=[(X_lgb_test, y_lgb_test)],early_stopping_rounds=100, verbose=False)
#     pred_lgb=lgbm.predict(X_lgb_test)
#     mae = mean_absolute_error(y_lgb_test, pred_lgb)
#     return mae


# study = optuna.create_study(
#     direction="minimize",
#     sampler=TPESampler(seed=42),
#     pruner=MedianPruner(n_warmup_steps=50),
# ) 

# study.optimize(objective_lgb, n_trials=100, show_progress_bar=True)

# study.best_trial.params

In [309]:
# def optimized_lgb(X_train1, y_train1, X_test1, y_test1):

#     # params = {
#     #     'objective': 'regression',
#     #     'metric': 'mae',
#     #     'boosting_type': 'rf',
#     #     'max_depth': 2,
#     #     'min_child_samples': 74,
#     #     'learning_rate': 0.4319464280296327,
#     #     'feature_fraction': 0.9,
#     #     'num_leaves': 680,
#     #     'bagging_freq': 1,
#     #     'bagging_fraction': 0.2,
#     #     'reg_alpha': 63.5,
#     #     'reg_lambda': 985,
#     #     'verbosity': -1,
#     # }

#     params = {
#         'boosting_type': 'gbdt',
#         'max_depth': 44,
#         'min_child_samples': 198,
#         'learning_rate': 0.24942129794855278,
#         'feature_fraction': 0.6000000000000001,
#         'num_leaves': 1160,
#         'bagging_freq': 1,
#         'bagging_fraction': 0.9,
#         'reg_alpha': 15.5,
#         'reg_lambda': 75,
#     }

#     lgbm = LGBMRegressor(**params, num_boost_round=1000)
#     lgbm.fit(X_train1, y_train1, eval_set=[(X_test1, y_test1)],early_stopping_rounds=100, verbose=False)
#     pred_lgb=lgbm.predict(X_test1)
#     mae = mean_absolute_error(y_test1, pred_lgb)
#     r2s = r2_score(y_test1, pred_lgb)
    
#     # y_pred = lgbm.predict(X_test1.values)
#     y_pred = pd.DataFrame(data=pred_lgb)
#     y_pred = sc_y.inverse_transform(y_pred)
    
#     return mae, r2s, y_pred

# optimized_lgb(X_lgb_train, y_lgb_train, X_lgb_test, y_lgb_test)

In [310]:
# from tensorboard.plugins.hparams import api as hp

# HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([16, 32, 64, 128]))
# HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([16, 32, 64, 128]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([16, 32, 64, 128]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))

# METRIC_ACCURACY = 'accuracy'

# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#   hp.hparams_config(
#     hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3],
#     metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
#   )

# def train_test_model(hparams):
#   model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation=tf.nn.relu),
#     tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation=tf.nn.relu),
#     tf.keras.layers.Dense(hparams[HP_NUM_UNITS3], activation=tf.nn.relu),
#     tf.keras.layers.Dense(1),
#   ])
#   model.compile(
#       optimizer='adam',
#       loss='mae',
#       metrics=['mae'],
#   )

#   model.fit(X_kbest_train, y_kbest_train, epochs=5) # Run with 1 epoch to speed things up for demo purposes
#   _, accuracy = model.evaluate(X_kbest_test, y_kbest_test)
#   return accuracy

# def run(run_dir, hparams):
#   with tf.summary.create_file_writer(run_dir).as_default():
#     hp.hparams(hparams)  # record the values used in this trial
#     accuracy = train_test_model(hparams)
#     tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

# session_num = 0

# for num_units1 in HP_NUM_UNITS1.domain.values:
#     for num_units2 in HP_NUM_UNITS2.domain.values:
#       for num_units3 in HP_NUM_UNITS3.domain.values:
#             hparams = {
#                 HP_NUM_UNITS1: num_units1,
#                 HP_NUM_UNITS2: num_units2,
#                 HP_NUM_UNITS3: num_units3
#             }
#             run_name = "run-%d" % session_num
#             print('--- Starting trial: %s' % run_name)
#             print({h.name: hparams[h] for h in hparams})
#             run('logs/hparam_tuning/' + run_name, hparams)
#             session_num += 1


In [311]:


# 'num_units1': 64, 'num_units2': 32, 'num_units3': 128

Reusing TensorBoard on port 6006 (pid 16000), started 16:58:01 ago. (Use '!kill 16000' to kill it.)

In [312]:
# # xgboost
# import xgboost

# def shap_xgbooster():
#     model = xgboost.XGBRegressor().fit(X, y)
#     explainer = shap.Explainer(model)
#     shap_values = explainer(X)

#     feature_names = shap_values.feature_names
#     shap_df = pd.DataFrame(shap_values.values, columns=feature_names)
#     vals = np.abs(shap_df.values).mean(0)
#     shap_importance = pd.DataFrame(list(zip(feature_names, vals)), columns=['col_name', 'feature_importance_vals'])
#     shap_importance.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
#     shap.plots.bar(shap_values)

#     return shap_importance.head(20)

# shap_xgbooster()

In [313]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# def objective_xgboost(trial):
    
#     params = {
#     'n_estimators':trial.suggest_int('n_estimators', 0, 100),
#     'verbosity': 0,
#     'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
#     'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
#     'min_child_weight':trial.suggest_int('min_child_weight', 0, 100), 
#     'gamma':trial.suggest_int('gamma', 0, 6),  
#     'subsample':trial.suggest_discrete_uniform('subsample', 0.1, 1, 0.01),
#     'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree', 0.1, 1, 0.01), 
#     'max_depth': trial.suggest_int('max_depth', 3, 100),
#     'objective': trial.suggest_categorical('objective', ['reg:squarederror']),
#     'booster': trial.suggest_categorical('booster',['gbtree', 'gblinear', 'dart']),
#     'eval_metric': ['logloss'],
#     'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1),
# }

#     model = xgboost.XGBRegressor(**params)
#     model.fit(X_shap_train,y_shap_train, eval_set=[(X_shap_test, y_shap_test)], early_stopping_rounds=50, verbose=False)
#     preds = model.predict(X_shap_test)
#     rmse = mean_squared_error(y_shap_test, preds, squared=False)

#     return rmse

# ntrial = 50
# study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42), pruner=MedianPruner(n_warmup_steps=50))
# study.optimize(objective_xgboost, n_trials=ntrial, show_progress_bar=True)

# print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [314]:
def optimized_xgboost(X_train1, y_train1, X_test1, y_test1, input_valid_shap):
    params = {'n_estimators': 100, 'reg_alpha': 3, 'reg_lambda': 0, 'min_child_weight': 29, 'gamma': 3, 'subsample': 0.99, 
    'colsample_bytree': 0.82, 'max_depth': 38, 'objective': 'reg:squarederror', 'booster': 'gbtree', 'learning_rate': 0.09140602270486103}
    model = xgboost.XGBRegressor(**params)
    model.fit(X_train1, y_train1)
    y_pred_xgboost = model.predict(input_valid_shap)
    y_estimate = model.predict(X_test1)
    r2s = r2_score(y_test1, y_estimate)
    mae = mean_absolute_error(y_test1, y_estimate)
    y_pred = sc_y.inverse_transform(y_pred_xgboost)

    return y_pred, r2s, mae

pred_xgboost = optimized_xgboost(X_shap_train, y_shap_train, X_shap_test, y_shap_test, input_valid_shap)
print(pred_xgboost)

solution_xg = pd.DataFrame(data=pred_xgboost[0], columns=['preds'])
solution_xg.to_csv('solutions/Regression_HauLamFong_xgboost.txt')

(array([ 85022.66 ,  85153.305, 233904.77 , ..., 123212.17 ,  83450.37 ,
       158029.22 ], dtype=float32), 0.9218083512407435, 0.14025287)


In [315]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# neural network

def optimized_nn(X_train1, y_train1, X_test1, y_test1, input_valid_kbest):
    model = Sequential([
        Dense(64,activation='relu',name='input_layer'),
        Dense(32,activation='relu',name='hidden_layer1'),
        Dense(128,activation='relu',name='hidden_layer2'),
        Dense(1, name='output')])

    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer,
                loss='mae',
                metrics='mse')

    print('--------- TRAINING ---------')
    history = model.fit(x=X_train1, y=y_train1, validation_data=(X_test1, y_test1), epochs = 10, batch_size=512)  
    score = model.evaluate(X_test1,  y_test1, verbose=2)
    y_estimate = model.predict(X_test1)
    y_pred_nn = model.predict(input_valid_kbest)
    mae = mean_absolute_error(y_test1, y_estimate)
    y_pred1 = sc_y.inverse_transform(y_pred_nn)

    return score, mae, y_pred1, r2_score(y_test1, y_estimate)

pred_nn = optimized_nn(X_kbest_train, y_kbest_train, X_kbest_test, y_kbest_test, input_valid_kbest)
print(pred_nn)

solution_nn = pd.DataFrame(data=pred_nn[2], columns=['preds'])
solution_nn.to_csv('solutions/Regression_HauLamFong_neuralnetwork.txt')

--------- TRAINING ---------
Epoch 1/10
190/190 [==============================] - 1s 2ms/step - loss: 0.1818 - mse: 0.1384 - val_loss: 0.1400 - val_mse: 0.0910
Epoch 2/10
190/190 [==============================] - 0s 2ms/step - loss: 0.1418 - mse: 0.1002 - val_loss: 0.1425 - val_mse: 0.0934
Epoch 3/10
190/190 [==============================] - 0s 2ms/step - loss: 0.1412 - mse: 0.0995 - val_loss: 0.1349 - val_mse: 0.0882
Epoch 4/10
190/190 [==============================] - 0s 1ms/step - loss: 0.1401 - mse: 0.0988 - val_loss: 0.1351 - val_mse: 0.0884
Epoch 5/10
190/190 [==============================] - 0s 1ms/step - loss: 0.1391 - mse: 0.0981 - val_loss: 0.1329 - val_mse: 0.0871
Epoch 6/10
190/190 [==============================] - 0s 1ms/step - loss: 0.1388 - mse: 0.0977 - val_loss: 0.1322 - val_mse: 0.0872
Epoch 7/10
190/190 [==============================] - 0s 1ms/step - loss: 0.1387 - mse: 0.0975 - val_loss: 0.1320 - val_mse: 0.0872
Epoch 8/10
190/190 [===========================

In [316]:
def optimized_lgb(X_train1, y_train1, X_test1, y_test1, input_valid_lgb):

    # params = {
    #     'objective': 'regression',
    #     'metric': 'mae',
    #     'boosting_type': 'rf',
    #     'max_depth': 2,
    #     'min_child_samples': 74,
    #     'learning_rate': 0.4319464280296327,
    #     'feature_fraction': 0.9,
    #     'num_leaves': 680,
    #     'bagging_freq': 1,
    #     'bagging_fraction': 0.2,
    #     'reg_alpha': 63.5,
    #     'reg_lambda': 985,
    #     'verbosity': -1,
    # }

    params = {
        'boosting_type': 'gbdt',
        'max_depth': 44,
        'min_child_samples': 198,
        'learning_rate': 0.24942129794855278,
        'feature_fraction': 0.6000000000000001,
        'num_leaves': 1160,
        'bagging_freq': 1,
        'bagging_fraction': 0.9,
        'reg_alpha': 15.5,
        'reg_lambda': 75,
    }

    lgb_train = lgb.Dataset(X_train1, y_train1)
    lgb_test = lgb.Dataset(X_test1, y_test1)

    lgbm = lgb.train(params, lgb_train,num_boost_round=1000, valid_sets=lgb_test, early_stopping_rounds=100)
    y_pred = lgbm.predict(input_valid_lgb)
    pred_lgb=lgbm.predict(X_test1)
    mae = mean_absolute_error(y_test1, pred_lgb)
    r2s = r2_score(y_test1, pred_lgb)
    
    # y_pred = lgbm.predict(X_test1.values)
    y_pred = sc_y.inverse_transform(y_pred)
    
    return mae, r2s, y_pred

pred_lgb = optimized_lgb(X_lgb_train, y_lgb_train, X_lgb_test, y_lgb_test, input_valid_lgb)
print(pred_lgb)

solution_lgb = pd.DataFrame(data=pred_lgb[2], columns=['preds'])
solution_lgb.to_csv('solutions/Regression_HauLamFong_lightgbm.txt')

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002359 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3060
[LightGBM] [Info] Number of data points in the train set: 97196, number of used features: 12
[LightGBM] [Info] Start training from score -0.000420
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l2: 0.599098
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l2: 0.382491
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l2: 0.264634
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l2: 0.190068
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's l2: 0.146842
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	val